# Week 7 Assignment

_MkKinney 6.1_

This week has been all about getting information off the internet both in structured data formats (CSV, JSON, etc) as well as HTML.  For these exercises, we're going to use two practical examples of fetching data from web pages to show how to use Pandas and BeautifulSoup to extract structured information from the web.

---
---

### 33.1 Parsing a list in HTML

Go to the Banner Health Price Transparency Page: https://www.bannerhealth.com/patients/billing/pricing-resources/hospital-price-transparency

Notice that there is a list of hospitals and the city they are in.  We want to parse the underlying HTML to create a list of all the hospitals along with which city they're in.

```json
[
    ["Banner - University Medical Center Phoenix", "Arizona"],
    ["Banner - University Medical Center South ", "Arizona"],
    ...
]
```

To examine the underlying HTML code, you can use Chrome, right-click, and choose **Inspect**.

For reference, the documentation for BeautifulSoup is here: https://www.crummy.com/software/BeautifulSoup/bs4/doc/


In [1]:
from bs4 import BeautifulSoup
import requests
headers = { "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36" }

response = requests.get('https://www.bannerhealth.com/patients/billing/pricing-resources/hospital-price-transparency', headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [2]:
div = soup.find_all('div', {"class":"col-md-8"})[0]
for hospital_list in div.find_all('ul'):
    state = hospital_list.previous_sibling.previous_sibling.string
    for hospital in hospital_list.find_all('li'):
        print(state, hospital.text)

Arizona Banner - University Medical Center Phoenix
Arizona Banner - University Medical Center South 
Arizona Banner - University Medical Center Tucson
Arizona Banner Baywood Medical Center 
Arizona Banner Behavioral Health Hospital
Arizona Banner Boswell Medical Center
Arizona Banner Casa Grande Medical Center
Arizona Banner Del E. Webb Medical Center
Arizona Banner Desert Medical Center/Cardon Children's Medical Center  
Arizona Banner Estrella Medical Center
Arizona Banner Gateway Medical Center/Banner MD Anderson Cancer Center
Arizona Banner Goldfield Medical Center  
Arizona Banner Heart Hospital
Arizona Banner Ironwood Medical Center
Arizona Banner Ocotillo Medical Center
Arizona Banner Payson Medical Center
Arizona Banner Thunderbird Medical Center
Arizona Page Hospital
California Banner Lassen Medical Center
Colorado Banner Fort Collins Medical Center
Colorado McKee Medical Center
Colorado North Colorado Medical Center
Colorado Sterling Regional Medical Center
Colorado East Morg

---

### 33.2 Using Pandas to Read Tables


Pandas documentation for loading data https://pandas.pydata.org/pandas-docs/version/0.23.4/api.html#input-output

Pandas documentation for describing the shape of data https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.shape.html

In [1]:
import pandas as pd

In [3]:
tables = pd.read_html('https://en.wikipedia.org/wiki/Diagnosis-related_group')
len(tables)
type(tables)

list

In [5]:
for index,table in enumerate(tables):
    print("**************TABLE {}".format(index))
    print(table)

**************TABLE 0
    0                                                  1
0 NaN  This article has multiple issues. Please help ...
1 NaN  This article needs to be updated. Please updat...
2 NaN  This article needs additional citations for ve...
**************TABLE 1
    0                                                  1
0 NaN  This article needs to be updated. Please updat...
**************TABLE 2
    0                                                  1
0 NaN  This article needs additional citations for ve...
**************TABLE 3
   Hypothetical patient at Generic Hospital in San Francisco, CA, DRG 482, HIP & FEMUR PROCEDURES EXCEPT MAJOR JOINT W/O CC/MCC (2001)[15]:8  \
0                                         Description                                                                                          
1                              Average length of stay                                                                                          
2                      L

In [6]:
drgs = tables[4]
drgs

,Name,Version,Start date,Notes
0,MS-DRG,25,"October 1, 2007","Group numbers resequenced, so that for instanc..."
1,MS-DRG,26,"October 1, 2008",One main change: implementation of Hospital Ac...
2,MS-DRG,27,"October 1, 2009",Changes involved are mainly related to Influen...
3,MS-DRG,31,"October 1, 2013",NaN
4,MS-DRG,32,"October 1, 2014",NaN
5,MS-DRG,33,"October 1, 2015",Convert from ICD-9-CM to ICD-10-CM.[17]
6,MS-DRG,34,"October 1, 2016",Address ICD-10 replication issues introduced i...
7,MS-DRG,35,"October 1, 2017",MS-DRGs 984 through 986 deleted and reassigned...


---

### 33.3 Find Something of Your Own

Do some web searches and find an HTML page with some data that is interesting to something you're studying.  You can extract and parse that information using either BeautifulSoup or Pandas.  If you're using Pandas, then do something interesting to format and structure your data.  If you're using BeautifulSoup, you'll just need to do the work of parsing the data out of HTML -- that's hard enough!

You don't need to build this as a function.  Just use notebook cells as I've done above.  You will be graded based on _style_.  Use variable names that make sense for your problem / solution. Cleanup anything you don't need before you submit your work.

In [7]:
# Your Code Here
import requests
from bs4 import BeautifulSoup
import json

In [8]:
#This will grab the treasury yield curve for the current month to date numbers. 
url = "https://www.treasury.gov/resource-center/data-chart-center/interest-rates/pages/textview.aspx?data=yield"
r = requests.get(url)
data = r.text

In [9]:
soup = BeautifulSoup(data)
table = soup.find('table',{'class':'t-chart'})

In [10]:
headers = []
for cell in table.tr.find_all('th'):
    headers.append(cell.text)
headers

['Date',
 '1 mo',
 '2 mo',
 '3 mo',
 '6 mo',
 '1 yr',
 '2 yr',
 '3 yr',
 '5 yr',
 '7 yr',
 '10 yr',
 '20 yr',
 '30 yr']

In [11]:
yields = {}
for row in table.find_all('tr'):
    colnum = 0
    for cell in row.find_all('td'):
        if colnum == 0:
            yld = cell.text
            yields.setdefault(yld,{})
        else:
            column = headers[colnum]
            yields[yld][column] = cell.text
        colnum += 1
print(json.dumps(yields,indent = 3))  

{
   "03/01/21": {
      "1 mo": "0.03",
      "2 mo": "0.03",
      "3 mo": "0.05",
      "6 mo": "0.07",
      "1 yr": "0.08",
      "2 yr": "0.13",
      "3 yr": "0.27",
      "5 yr": "0.71",
      "7 yr": "1.12",
      "10 yr": "1.45",
      "20 yr": "2.11",
      "30 yr": "2.23"
   },
   "03/02/21": {
      "1 mo": "0.04",
      "2 mo": "0.04",
      "3 mo": "0.04",
      "6 mo": "0.06",
      "1 yr": "0.08",
      "2 yr": "0.13",
      "3 yr": "0.26",
      "5 yr": "0.67",
      "7 yr": "1.08",
      "10 yr": "1.42",
      "20 yr": "2.09",
      "30 yr": "2.21"
   },
   "03/03/21": {
      "1 mo": "0.04",
      "2 mo": "0.04",
      "3 mo": "0.05",
      "6 mo": "0.07",
      "1 yr": "0.08",
      "2 yr": "0.14",
      "3 yr": "0.29",
      "5 yr": "0.73",
      "7 yr": "1.14",
      "10 yr": "1.47",
      "20 yr": "2.12",
      "30 yr": "2.25"
   },
   "03/04/21": {
      "1 mo": "0.03",
      "2 mo": "0.05",
      "3 mo": "0.04",
      "6 mo": "0.07",
      "1 yr": "0.08",
    

In [12]:
#This prints out the options for the dropdown menu for the time periods
year = soup.find('select', id = 'interestRateTimePeriod')
for item in year:
    print(item.text)

Current Month
2021
2020
2019
2018
2017
2016
2015
2014
2013
2012
2011
2010
2009
2008
2007
2006
2005
2004
2003
2002
2001
2000
1999
1998
1997
1996
1995
1994
1993
1992
1991
1990
		    
	  All


In [13]:
#How would I be able to select a certain time period? The default is the current month, but I would like to be able to select the current year.

---

## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Save this note with Ctrl-S (or Cmd-S)
2. Skip down to the last command cell (the one starting with `%%bash`) and run that cell.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

In [14]:
assert False, "DO NOT REMOVE THIS LINE"

AssertionError: DO NOT REMOVE THIS LINE

---

In [18]:
%%bash
git pull
git add week08_assignment_2.ipynb
git commit -a -m "Submitting the week 8 programming assignment"
git push

Already up to date.
On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
	../week02/week02_inclass.ipynb
	../week04/week04_lookups.ipynb

nothing added to commit but untracked files present


Everything up-to-date



---

If the message above says something like _Submitting the week 8 programming assignment_ or _Everything is up to date_, then your work was submitted correctly.